In [1]:
from ipyleaflet import Map, TileLayer, LayersControl, basemaps, basemap_to_tiles, WidgetControl
import ipywidgets as widgets
from ipywidgets import Layout
from IPython.display import display
import requests

In [2]:
chosen_basemap = basemap_to_tiles(basemaps.OpenStreetMap.Mapnik)
chosen_basemap.name = '(basemap) OpenStreetMap Mapnik'
# Need to find a way to programmatically get center for diff locations
m = Map(center=(13.478, -15.125), zoom=6)

In [3]:
# Group layers for easier visualization in map control
water_risk_layers = []
population_risk_layers = []
absolute_risk_layers = []
relative_risk_layers = []
# Map layer names to TileLayer representation
checkbox_layers_map = {}

In [5]:
layers_index_path = 'http://localhost:8080/output/jupyter-layers.txt'

page = requests.get(layers_index_path).text
layer_name_list = []
for line in page.splitlines():
    layer_name_list.append(line + "_tiles")
#print(layer_name_list)


['rural_urbanization_suit_tiles', 'snail_water_temp_suit_dry_tiles', 'parasite_water_temp_suit_dry_tiles', 'ndvi_suit_dry_tiles', 'snail_water_temp_suit_wet_tiles', 'parasite_water_temp_suit_wet_tiles', 'ndvi_suit_wet_tiles', 'habitat_suit_weighted_mean_tiles', 'convolved_hab_risk_tiles', 'normalized_convolved_risk_tiles', 'risk_to_pop_abs_tiles', 'risk_to_pop_count_abs_tiles', 'risk_to_pop_rel_tiles', 'risk_to_pop_count_rel_tiles']


In [6]:
# Add NDVI layers
# TODO: how do we know which layer have been run?
# Schisto could save a output/layers.txt we could read, which identifies the tile folders...
for season in ['dry', 'wet']:
    # only display layers from the most recent model run
    base_tile_dir = f'ndvi_suit_{season}_tiles'
    if base_tile_dir in layer_name_list:
        ndvi_tile_url = TileLayer(
            url=f'http://localhost:8080/output/{base_tile_dir}/{{z}}/{{x}}/{{y}}.png',
            name=f"ndvi {season}",
            attribution="mine", #TODO: do we need anything here?
            min_zoom=1,
            max_zoom=16,
            min_native_zoom=1,
            max_native_zoom=14,)
        water_risk_layers.append(f'ndvi_suit_{season}')
        checkbox_layers_map[f'ndvi_suit_{season}'] = ndvi_tile_url

In [7]:
# Add water temperature layers
for risk_type in ['parasite', 'snail']:
    for season in ['dry', 'wet']:
        # only display layers from the most recent model run
        base_tile_dir = f'{risk_type}_water_temp_suit_{season}_tiles'
        if base_tile_dir in layer_name_list:
            temp_tile_url = TileLayer(
                url=f'http://localhost:8080/output/base_tile_dir/{{z}}/{{x}}/{{y}}.png',
                name=f"{risk_type} water temp {season}",
                attribution="mine",
                min_zoom=1,
                max_zoom=16,
                min_native_zoom=1,
                max_native_zoom=14,)
            water_risk_layers.append(f'{risk_type}_water_temp_suit_{season}')
            checkbox_layers_map[f'{risk_type}_water_temp_suit_{season}'] = temp_tile_url

In [8]:
# Add population risk layers
population_risk_tile_url = TileLayer(
    url='http://localhost:8080/output/rural_urbanization_suit_tiles/{z}/{x}/{y}.png',
    name="Population risk",
    attribution="mine",
    min_zoom=1,
    max_zoom=16,
    min_native_zoom=1,
    max_native_zoom=14,)
population_risk_layers.append('rural_urbanization_suit')
checkbox_layers_map['rural_urbanization_suit'] = population_risk_tile_url

In [9]:
# Add weighted mean risk layer
combined_risk_tile_url = TileLayer(
    url='http://localhost:8080/output/habitat_suit_weighted_mean_tiles/{z}/{x}/{y}.png',
    name="Combined risk",
    attribution="mine",
    min_zoom=1,
    max_zoom=16,
    min_native_zoom=1,
    max_native_zoom=14,)
water_risk_layers.append('habitat_suit_weighted_mean')
checkbox_layers_map['habitat_suit_weighted_mean'] = combined_risk_tile_url

In [10]:
# Add distance weighted risk (convolution layer)
convolved_risk_tile_url = TileLayer(
    url='http://localhost:8080/output/convolved_hab_risk_tiles/{z}/{x}/{y}.png',
    name="Distance weighted risk",
    attribution="mine",
    min_zoom=1,
    max_zoom=16,
    min_native_zoom=1,
    max_native_zoom=14,)
absolute_risk_layers.append('convolved_hab_risk')
checkbox_layers_map['convolved_hab_risk'] = convolved_risk_tile_url

In [11]:
# Add normalized distance weighted risk
normalized_risk_tile_url = TileLayer(
    url='http://localhost:8080/output/normalized_convolved_risk_tiles/{z}/{x}/{y}.png',
    name="Distance weighted risk",
    attribution="mine",
    min_zoom=1,
    max_zoom=16,
    min_native_zoom=1,
    max_native_zoom=14,)
relative_risk_layers.append('normalized_convolved_risk')
checkbox_layers_map['normalized_convolved_risk'] = normalized_risk_tile_url

In [12]:
# Add relative and absolute risk to population outputs
for risk_type, layer_group in zip(['abs', 'rel'], [absolute_risk_layers, relative_risk_layers]):
    risk_to_pop_tile_url = TileLayer(
        url=f'http://localhost:8080/output/risk_to_pop_{risk_type}_tiles/{{z}}/{{x}}/{{y}}.png',
        name=f"Risk to pop {risk_type}",
        attribution="mine",
        min_zoom=1,
        max_zoom=16,
        min_native_zoom=1,
        max_native_zoom=14,)
    layer_group.append(f'risk_to_pop_{risk_type}')
    checkbox_layers_map[f'risk_to_pop_{risk_type}'] = risk_to_pop_tile_url
    
    risk_to_pop_count_tile_url = TileLayer(
        url=f'http://localhost:8080/output/risk_to_pop_count_{risk_type}_tiles/{{z}}/{{x}}/{{y}}.png',
        name=f"Risk to pop count {risk_type}",
        attribution="mine",
        min_zoom=1,
        max_zoom=16,
        min_native_zoom=1,
        max_native_zoom=14,)
    layer_group.append(f'risk_to_pop_count_{risk_type}')
    checkbox_layers_map[f'risk_to_pop_count_{risk_type}'] = risk_to_pop_count_tile_url

In [13]:
out = widgets.Output()

@out.capture()
def layer_visible_switch(event):
    widget = event['owner']
    desc_id = widget.description
    print(desc_id)
    layer_tile_url = checkbox_layers_map[desc_id]
    layer_tile_url.visible = event['new']

layer_groups = [water_risk_layers, population_risk_layers, absolute_risk_layers, relative_risk_layers]
layer_group_sections = ["Water risk layers", "Population risk layers", "Absolute risk layers", "Relative risk layers"]
widget_list = []
for group_title, group_list in zip(layer_group_sections, layer_groups):
    group_header_widget = widgets.HTML(description="", value=f'<b>{group_title}<b>')
    checkbox_widget_group = []
    for layer_key in group_list:
        tmp_widget = widgets.Checkbox(
                value=False,
                description=layer_key,
                disabled=False,
                indent=False,
                layout=Layout(padding='0px 0px 0px 10px', margin='0px 0px 0px 0px', width='auto'),
            )
        tmp_widget.observe(layer_visible_switch, names='value')
        checkbox_widget_group.append(tmp_widget)
        checkbox_layers_map[layer_key].visible=False
        m.add(checkbox_layers_map[layer_key])
    
    widget_list.append(widgets.Accordion(children=[widgets.VBox(checkbox_widget_group)], titles=(group_title,)))
    
vbox_widget = widgets.VBox(widget_list, layout=Layout(height='auto', width='auto', overflow_y='auto', padding='0px 0px 0px 0px'))
layers_control_acc = widgets.Accordion(children=[vbox_widget], titles=("Layers",), layout=Layout(max_height='250px', padding='0px 0px 0px 0px'))


In [14]:
widget_control = WidgetControl(widget=layers_control_acc, position="bottomleft")

In [15]:
m.add(widget_control)

Map(center=[13.478, -15.125], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', 'zoo…

In [ ]:
# use the LayersControl to switch basemaps
#m.add_control(LayersControl())

In [ ]:
import matplotlib.pyplot as plt
from imageio.v2 import imread

plot_index_path = 'http://localhost:8080/intermediate/plot_previews/index.txt'

def get_files(url):
    plot_root_path = 'http://localhost:8080/intermediate/plot_previews/'
    page = requests.get(url).text
    plot_png_list = []
    for line in page.splitlines():
        if ('png' in line.lower()):
            plot_png_list.append(plot_root_path + line)
    return plot_png_list

onlyfiles = get_files(plot_index_path)  

In [ ]:
import math
n_rows = math.ceil(len(onlyfiles) / 3)
fig_size = (19, 12)
fig, axs = plt.subplots(nrows=n_rows, ncols=3, figsize=fig_size,
                        subplot_kw={'xticks': [], 'yticks': []})

fig.suptitle("Suitability function plots")
for ax, image_file in zip(axs.flat, onlyfiles):
    url_img = imread(image_file)
    ax.imshow(url_img)
    ax.set_aspect('equal')

hide_ax_index = len(onlyfiles) % 3
if hide_ax_index != 0:
    for ax_index in range(hide_ax_index,3):
        fig.delaxes(axs[n_rows-1][ax_index])
plt.tight_layout()
plt.show()
